# TP 2 : Régression logistique

## 1 Régularisation de Tikhonov

### 1.1 : Étude de la fonction objectif :

**Calcul du gradient :**

$\frac{df_1}{dw_0} (w_0, w)= \frac 1 n \sum_{i=1}^n{\frac{-y_i}{1+exp(y_i(x_i^T*w + w_0))}}$


$\frac{df_1}{dw} (w_0, w)= \frac 1 n \sum_{i=1}^n{\frac{-y_i*x_i}{1+exp(y_i(x_i^T*w + w_0))}} + \rho * w$

**Calcul de la hessienne :**

$H = \frac 1 n * \sum_i{\frac{y_i^2}{(2cosh(y_i/2(x_i^T*w + w_0))^2}} * \begin{bmatrix}1 \\ x_i \end{bmatrix} \begin{bmatrix}1 \\ x_i \end{bmatrix}^T + \begin{vmatrix} 0 & 0 \\ 0 & {\rho * I_p} \end{vmatrix}  $

Montrons que $H$ est définie positive. Posons $a_i = \frac{y_i}{\sqrt n (2cosh(y_i/2(x_i^T*w + w_0))}$ et $v_i = \begin{bmatrix}1 \\ x_i \end{bmatrix}$, on a :

$\forall u \in \mathbb R^{p+1}, u^T H u = u^T \begin{vmatrix} 0 & 0 \\ 0 & {\rho * I_p} \end{vmatrix} u + \sum_i{(u^T a_i v_i) (v_i^T a_i u) }$

* $u^T \begin{vmatrix} 0 & 0 \\ 0 & {\rho * I_p} \end{vmatrix} u = \begin{bmatrix} 0 & {\rho \sum_{i=2}^{p+1}{u_i}} & {...} & {\rho \sum_{i=2}^{p+1}{u_i}} \end{bmatrix} \begin{bmatrix} u_1 \\ ... \\ u_ {p+1} \end{bmatrix} = \rho (\sum_{i=2}^{p+1}{u_i})^2 > 0$
* $\sum_i{(u^T a_i v_i) (v_i^T a_i u) } = \sum_i{(u^T a_i v_i)^2 } > 0$

Ainsi $ u^T H u > 0$. 

Et donc $f_1$ est convexe

### 1.2 : Programmation :

In [37]:
import numpy as np
from scipy.optimize import check_grad

def objective(w, X, Y, rho, verbose=False):
    """ Renvoie les données numériques du problème
    :param w: Le vecteur [w0 w] de l'énoncé. On attend une liste de longueur p+1.
    :param X: La matrice des vecteurs lignes xi. On attend une matrice de dimension n lignes * p+1 colonnes.
        La 1ère colonne est composée uniquement de 1.
    :param Y: Le vecteur des yi. Liste de longueur n.
    :param rho: Le paramètre de régularisation.
    
    :return: La valeur de la fonction, son gradient et sa hessienne."""
    assert (len(Y), len(w)) == X.shape, "Erreur de dimensions"
    n = len(Y)
    p = len(w) - 1
    # Valeur de la fonction au point w :
    val = rho/2 * np.linalg.norm(w, 2)**2
    for i in range(n):
        s = np.exp(-Y[i]*np.dot(X[i,:], w))
        if verbose:
            print("Étape %d, Val = %f" % (i, val))
            print("s:", s)
        val += 1/n*np.log(1 + s)
    
    # Gradient de la fonction au point w 
    grad = np.zeros(p+1)
    grad[1:] = rho * w[1:]
    for i in range(n):
        grad += 1/n * (Y[i]*X[i])/(1 + np.exp(Y[i] * np.dot(X[i,:], w)))
        
    # Matrice hessienne :
    hess = np.identity(p+1)*rho
    hess[0,0] = 0
    for i in range(n):
        v = X[i,:][np.newaxis]
        hess += 1/n * (Y[i] / (2*np.cosh(Y[i]/(2*np.dot(X[i,:], w)))))**2 * v.T.dot(v)
#         if verbose:
#             print("v.T.dot(v):")
#             print(v.T.dot(v))
#             print("np.dot(X[i,:], w) = %f" % np.dot(X[i,:], w))
#             print("(Y[i] / (2*np.cosh(Y[i]/(2*np.dot(X[i,:], w)))))**2 = %f"% 
#                   (Y[i] / (2*np.cosh(Y[i]/(2*np.dot(X[i,:], w)))))**2)
#             print("hess:")
#             print(hess)
    return val, grad, hess


f(w) = 5.533369, attendu : 5.283369
Écart entre la hessienne et la valeur attendue :
[[ 0.  0.  0.]
 [ 0.  0.  0.]
 [ 0.  0.  0.]]


In [42]:
from cervicalcancerutils import load_cervical_cancer


# Test unitaire :
y_test = np.array([-1, 1])
X_test = np.array([[1,0,1],[1,1,0]])
w_test = np.array([1,2,3])
rho_test = 1/2

print("Test unitaire :")
# f(w) :
val_expected = 1/2*(np.log(1+np.exp(4)) + np.log(1+np.exp(-3))) + rho_test/2 * 13
# hessienne :
a1 = 1/2 * (1/(2*np.cosh(-1/8)))**2
a2 = 1/2 * (1/(2*np.cosh(1/6)))**2
m1 = np.array([[1,0,1], [0,0,0], [1,0,1]])
m2 = np.array([[1,1,0], [1,1,0], [0,0,0]])
m3 = np.array([[0,0,0], [0,1,0], [0,0,1]]) * rho_test
hess_expected = a1*m1 + a2*m2 + m3


val, grad, hess = objective(w_test, X_test, y_test, rho_test, verbose=False)
print("f(w) = %f, attendu : %f" % (val, val_expected))
print("Écart entre la hessienne et la valeur attendue :")
print(hess - hess_expected)

X,y = load_cervical_cancer("riskfactorscervicalcancer.csv")
(n, p) = X.shape

# On rajoute une colonne de 1 à X :
X = np.hstack((np.ones((X.shape[0], 1)), X))

# Affichage des erreurs sur le gradient :
n_verif = 10 # Nombre de vérification à effectuer
print("Pour le gradient, avec les données 'cervical_cancer' :")
print("Écart absolu | Écart relatif")
print("----------------------------")
for i in range(n_verif):
    rho = np.random.random()
    w = np.random.random(p+1)
    f = lambda w:objective(w, X, y, rho)[0]
    grad_f = lambda w:objective(w, X, y, rho)[1]
    err = check_grad(f, grad_f, w)
    err_rel = err/np.linalg.norm(grad_f(w))
    print("%10.3f   | %5.1f%%" % (err, err_rel))

Test unitaire :
f(w) = 5.533369, attendu : 5.283369
Écart entre la hessienne et la valeur attendue :
[[ 0.  0.  0.]
 [ 0.  0.  0.]
 [ 0.  0.  0.]]
Pour le gradient, avec les données 'cervical_cancer' :
Écart absolu | Écart relatif
----------------------------
     0.834   |   0.4%
     0.891   |   0.4%
     0.938   |   0.6%
     0.913   |   3.5%
     1.152   |   0.5%
     0.870   |   0.5%
     1.013   |   0.8%
     0.924   |   2.3%
     0.908   |   0.5%
     0.950   |   0.5%


In [44]:
print(np.linalg.inv(hess_expected))
print(np.linalg.inv(hess))

[[ 5.08736545 -0.99515305 -1.00484695]
 [-0.99515305  1.80343922  0.19656078]
 [-1.00484695  0.19656078  1.80343922]]
[[ 5.08736545 -0.99515305 -1.00484695]
 [-0.99515305  1.80343922  0.19656078]
 [-1.00484695  0.19656078  1.80343922]]


### 1.3 : Méthode de Newton :

In [52]:
def newton(objective, start, epsilon=0.1, max_iter=1000,verbose=False):
    """ Renvoie le point qui minimise la fonction objectif.
    :param objective: une fonction qui prend un vecteur et renvoie (valeur, gradient, hessienne) en ce point.
    :param start: le point de départ
    
    :return: le point qui minimise la fonction.
    """
    
    i = 1
    
    proceed = True
    i = 1
    point = start
    while proceed:
        val, grad, hess = objective(point)
        norm_grad = np.linalg.norm(grad)
        if verbose:
            print("Itération %d, val = %f, ||grad|| = %f" % (i, val, norm_grad))
        point -= np.linalg.inv(hess).dot(grad)
        i += 1
        proceed = i <= max_iter and norm_grad < epsilon

optimum = newton(lambda w:objective(w, X, y, rho), np.zeros_like(w), verbose=True)

Itération 1, val = 0.693147, ||grad|| = 0.348264


/home/loic/.local/lib/python3.5/site-packages/ipykernel/__main__.py:36: RuntimeWarning: divide by zero encountered in double_scalars


LinAlgError: Singular matrix

In [59]:
val, grad, hess = objective(np.zeros_like(w), X, y, 1/n)

/home/loic/.local/lib/python3.5/site-packages/ipykernel/__main__.py:36: RuntimeWarning: divide by zero encountered in double_scalars


In [60]:
print(val)
print(grad)
print(hess)

0.69314718056
[-0.01428571  0.02614069 -0.00519101 -0.00713406  0.03676745  0.02519034
  0.04564283  0.01268279  0.04115343  0.08619172  0.07996712  0.06396583
  0.10082943  0.10233987  0.09926846  0.09926846 -0.01338632  0.03557389
  0.07747075 -0.03359756  0.10242157  0.03330453  0.03178166  0.10882475
  0.03682732  0.10882475  0.09953978]
[[ 0.         0.         0.         0.         0.         0.         0.         0.
   0.         0.         0.         0.         0.         0.         0.         0.
   0.         0.         0.         0.         0.         0.         0.         0.
   0.         0.         0.       ]
 [ 0.         0.0047619  0.         0.         0.         0.         0.         0.
   0.         0.         0.         0.         0.         0.         0.         0.
   0.         0.         0.         0.         0.         0.         0.         0.
   0.         0.         0.       ]
 [ 0.         0.         0.0047619  0.         0.         0.         0.         0.
   